In [1]:
from helpers import make_integer, get_detections, get_iou
            
def get_color(tracks, dicto):
    i = 0
    j = i + 1
    
    
    # This is written to counter the problem of track_ids remaining in dictionary because of identity switching. 
    #The below code removes the dictionary entries containing track_ids which do not exist in the current frame.
    current_track_ids = [lst[-1] for lst in tracks]
    for trackids in list(dicto.keys()):
        if trackids[0] not in current_track_ids or trackids[1] not in current_track_ids:
            del dicto[trackids]
    
     
    #print('len(tracks): ', len(tracks))
    while i != len(tracks) - 2 and len(tracks):
        #print('i: ', i)
        #print('j: ', j)
        #print("==========")
        if j == len(tracks):
            i += 1
            j = i + 1
            
        
        b1 = tracks[i]
        b2 = tracks[j]
        
        bbox1 = {'x1': b1[0], 'y1': b1[1],'x2': b1[2],'y2': b1[3]}
        bbox2 = {'x1': b2[0], 'y1': b2[1],'x2': b2[2],'y2': b2[3]}
        track1 = int(b1[4])
        track2 = int(b2[4])
        iou = get_iou(bbox1, bbox2)
        #print(iou)================
        if iou > 0:
            if (track1,track2) in dicto.keys():
                dicto[(track1, track2)] += 1
            else:
                dicto.update({(track1, track2) : 1})
        elif iou == 0: #Since no overlap, 
            if (track1,track2) in dicto.keys():
                del dicto[(track1, track2)]
        
        j += 1
        
    
    return dicto

In [2]:
import cv2
import numpy as np
import imutils
import time
# reading videos with imutils
from imutils.video import FileVideoStream
from imutils.video import FPS
import torch
from sort.sort import *
import concurrent.futures as cf

#create instance of SORT
mot_tracker = Sort() 

path1 = "PETS2009 1.avi"

# Loading yolov5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained = True)

model.eval()

model.conf = 0.50

model.classes = [0]

c = 0
dicto = {}
fm1 = []
fm5 = []
video = FileVideoStream(path1).start()
fps = FPS().start()
time.sleep(1.0)
fm =0
while video.more():
    frame = video.read()
    if frame is None:
        break

    fm1 = []
    fm5 = []
    
    preds = get_detections(model, frame)
    
    
    dets = preds.pred[0]
    
    #det = dets[0].unsqueeze(0)
    
    #print(det)
    # update SORT
    track_bbs_ids = mot_tracker.update(dets)
    
    '''
    # Implement async
    with cf.ThreadPoolExecutor(max_workers = 4) as executor:
        results = list(executor.map(get_color, track_bbs_ids, dicto, timeout = 3))
    
    print('results: ', results)
    c += 1
    if c == 50:
        break
    

    '''
    
    dicto = get_color(track_bbs_ids, dicto)
    
    
    
    
    #print("======")
    for tracks, fm_no in dicto.items():
        if fm_no >= 5:
            print(f"5-FRAME Alert for {(tracks[0], tracks[1])}")
            fm5.append(tracks[0])
            fm5.append(tracks[1])
            
        elif fm_no >= 1 and fm_no < 5:
            print(f"1-FRAME Alert for {(tracks[0], tracks[1])}")
            fm1.append(tracks[0])
            fm1.append(tracks[1])
            
        
    #print("fm1: ", fm1)  
    #print("fm5: ", fm5)
    #print("================")
    
    for box in track_bbs_ids:
        track_id = box[4]
        if track_id in fm1:
            color = (255,0,0)
            
        elif track_id in fm5:
            color = (0,0,255)
        else:
            color = (0,255,255)
            
        x1,y1,x2,y2 = make_integer(box[:4])
        
        cv2.rectangle(frame, (x1,y1), (x2,y2), color, 1)
        cv2.putText(frame, str(int(track_id)),(x1+4, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1)
        
        #time.sleep(0.5)

    c += 1
    if c==500:
        break
    
   
    #display
    cv2.imshow('Video', frame)
    cv2.waitKey(1)
    fps.update()
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# stop the timer and display FPS information
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
video.stop()    
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.waitKey(1)


Using cache found in /Users/sumit/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-1-24 Python-3.8.8 torch-1.9.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


1-FRAME Alert for (6, 5)
1-FRAME Alert for (6, 5)
1-FRAME Alert for (6, 5)
1-FRAME Alert for (6, 5)
5-FRAME Alert for (6, 5)
5-FRAME Alert for (6, 5)
1-FRAME Alert for (7, 1)
5-FRAME Alert for (6, 5)
5-FRAME Alert for (6, 5)
5-FRAME Alert for (6, 5)
5-FRAME Alert for (6, 5)
5-FRAME Alert for (6, 5)
1-FRAME Alert for (8, 1)
5-FRAME Alert for (6, 5)
5-FRAME Alert for (6, 5)
5-FRAME Alert for (6, 5)
5-FRAME Alert for (6, 5)
5-FRAME Alert for (6, 5)
5-FRAME Alert for (6, 5)
5-FRAME Alert for (6, 5)
1-FRAME Alert for (9, 1)
5-FRAME Alert for (6, 5)
1-FRAME Alert for (9, 1)
5-FRAME Alert for (6, 5)
1-FRAME Alert for (9, 1)
5-FRAME Alert for (6, 5)
1-FRAME Alert for (9, 1)
5-FRAME Alert for (6, 5)
5-FRAME Alert for (9, 1)
5-FRAME Alert for (6, 5)
5-FRAME Alert for (9, 1)
5-FRAME Alert for (6, 5)
5-FRAME Alert for (6, 5)
1-FRAME Alert for (12, 9)
1-FRAME Alert for (12, 9)
1-FRAME Alert for (12, 9)
1-FRAME Alert for (6, 5)
1-FRAME Alert for (12, 9)
1-FRAME Alert for (6, 5)
5-FRAME Alert for (12

-1